In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# importo los .py
import sys
sys.path.append("auxiliares")
import config
import feature_eng

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
#Manual
prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_manual_"
prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_manual_"
num_clusters=94

#Time Warping
#prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_warp_"
#prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_warp_"

arch_productos = "datasets/emp3_sellout_producto_base.csv"
arch_clientes = "datasets/emp3_sellout_cliente_base.csv"
arch_cat1 = "datasets/emp3_sellout_cat1_base.csv"
arch_cat2 = "datasets/emp3_sellout_cat2_base.csv"
arch_cat3 = "datasets/emp3_sellout_cat3_base.csv"

In [4]:
df_productos = pd.read_csv(arch_productos)
df_productos = df_productos.rename(columns={"product_id_tn":"tn"})
df_productos = df_productos[["product_id","periodo","tn"]] # limito solamente a columnas de interes
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34707 entries, 0 to 34706
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  34707 non-null  int64  
 1   periodo     34707 non-null  int64  
 2   tn          34707 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 813.6 KB


In [5]:
df_clientes = pd.read_csv(arch_clientes)
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18975 entries, 0 to 18974
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customer_id     18975 non-null  int64  
 1   periodo         18975 non-null  int64  
 2   customer_id_tn  18975 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 444.9 KB


In [6]:
df_cat1 = pd.read_csv(arch_cat1)
df_cat1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cat1     140 non-null    object 
 1   periodo  140 non-null    int64  
 2   cat1_tn  140 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.4+ KB


In [7]:
df_cat2 = pd.read_csv(arch_cat2)
df_cat2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cat2     560 non-null    object 
 1   periodo  560 non-null    int64  
 2   cat2_tn  560 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.2+ KB


In [8]:
df_cat3 = pd.read_csv(arch_cat3)
df_cat3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717 entries, 0 to 2716
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cat3     2717 non-null   object 
 1   periodo  2717 non-null   int64  
 2   cat3_tn  2717 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 63.8+ KB


In [9]:
for i in range(0,num_clusters):
    print("Cluster", i)
    
    df_sellout = pd.read_csv(prefijo_arch_entrada + str(i) + ".csv")
    
    #Variable Target
    df_sellout = feature_eng.crear_target_prod_cust(df_sellout)
    
    #Nuevos Features
    df_sellout = feature_eng.crear_nuevos_features(df_sellout)
    
    #Features temporales
    df_sellout = feature_eng.crear_features_temporales("prod_cust",df_sellout, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_productos = feature_eng.crear_features_temporales("product_id",df_productos,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_clientes = feature_eng.crear_features_temporales("customer_id",df_clientes,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat1 = feature_eng.crear_features_temporales("cat1",df_cat1,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat2 = feature_eng.crear_features_temporales("cat2",df_cat2, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat3 = feature_eng.crear_features_temporales("cat3",df_cat3, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)

    #Joins
    print(len(df_sellout))
    df_productos_para_merge = df_productos.rename(columns={"tn":"product_id_tn"}) #revierto cambio para evitar colisiones

    if "product_id" in config.AMPLIA_FEATURES_PARAM:
        df_sellout = df_sellout.merge(df_productos_para_merge,how="left",on=["product_id","periodo"])

    if "customer_id" in config.AMPLIA_FEATURES_PARAM:
       df_sellout = df_sellout.merge(df_clientes,how="left",on=["customer_id","periodo"])

    if "cat1" in config.AMPLIA_FEATURES_PARAM:
        df_sellout = df_sellout.merge(df_cat1,how="left",on=["cat1","periodo"])
    
    if "cat2" in config.AMPLIA_FEATURES_PARAM:
        df_sellout = df_sellout.merge(df_cat2,how="left",on=["cat2","periodo"])
    
    if "cat3" in config.AMPLIA_FEATURES_PARAM:
        df_sellout = df_sellout.merge(df_cat3,how="left",on=["cat3","periodo"])
    print(len(df_sellout))
    
    df_sellout.to_csv(prefijo_arch_salida + str(i) + ".csv", index=False)

Cluster 0
867137
867137
Cluster 1
625413
625413
Cluster 2
229629
229629
Cluster 3
1736970
1736970
Cluster 4
199109
199109
Cluster 5
769474
769474
Cluster 6
246675
246675
Cluster 7
187185
187185
Cluster 8
87708
87708
Cluster 9
142584
142584
Cluster 10
331448
331448
Cluster 11
37950
37950
Cluster 12
418835
418835
Cluster 13
2133450
2133450
Cluster 14
170775
170775
Cluster 15
18975
18975
Cluster 16
85562
85562
Cluster 17
121805
121805
Cluster 18
116629
116629
Cluster 19
75900
75900
Cluster 20
155085
155085
Cluster 21
75900
75900
Cluster 22
319803
319803
Cluster 23
37950
37950
Cluster 24
67373
67373
Cluster 25
67373
67373
Cluster 26
1609045
1609045
Cluster 27
94875
94875
Cluster 28
41312
41312
Cluster 29
44138
44138
Cluster 30
234922
234922
Cluster 31
426605
426605
Cluster 32
181964
181964
Cluster 33
27054
27054
Cluster 34
99414
99414
Cluster 35
799560
799560
Cluster 36
136635
136635
Cluster 37
398475
398475
Cluster 38
38834
38834
Cluster 39
186996
186996
Cluster 40
189737
189737
Cluster 4

In [10]:
df_sellout

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,promedio_mens_dolar_venta,catastrofe,tn_mas_2,dif_cust_request_tn,dif_cust_request_tn_porc,prod_cust_tn_lag_1,prod_cust_tn_lag_2,prod_cust_tn_lag_3,prod_cust_tn_lag_4,prod_cust_tn_lag_5
0,21264,201707,10001,0,0.0,1.0,0.00780,0.00780,2017-07-01,7,...,17.19,False,0.00298,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,21264,201708,10001,0,0.0,3.0,0.00593,0.00593,2017-08-01,8,...,17.42,False,0.00556,0.0,0.0,0.00780,NaN,NaN,NaN,NaN
2,21264,201709,10001,0,0.0,4.0,0.00298,0.00298,2017-09-01,9,...,18.15,False,0.00223,0.0,0.0,0.00593,0.00780,NaN,NaN,NaN
3,21264,201710,10001,0,0.0,4.0,0.00556,0.00556,2017-10-01,10,...,17.46,False,0.00075,0.0,0.0,0.00298,0.00593,0.00780,NaN,NaN
4,21264,201711,10001,0,0.0,2.0,0.00223,0.00223,2017-11-01,11,...,17.48,False,0.00149,0.0,0.0,0.00556,0.00298,0.00593,0.0078,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32017,21271,201907,10637,1,0.0,0.0,0.00000,0.00000,2019-07-01,7,...,42.58,False,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.0000,0.0
32018,21271,201909,10637,1,0.0,0.0,0.00000,0.00000,2019-09-01,9,...,56.46,False,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.0000,0.0
32019,21271,201910,10637,1,0.0,0.0,0.00000,0.00000,2019-10-01,10,...,58.54,False,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.0000,0.0
32020,21271,201911,10637,1,0.0,0.0,0.00000,0.00000,2019-11-01,11,...,59.75,False,NaN,0.0,0.0,0.00000,0.00000,0.00000,0.0000,0.0


## Verificaciones

In [11]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32022 entries, 0 to 32021
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 32022 non-null  int64  
 1   periodo                    32022 non-null  int64  
 2   customer_id                32022 non-null  int64  
 3   cero_ventas                32022 non-null  int64  
 4   plan_precios_cuidados      32022 non-null  float64
 5   cust_request_qty           32022 non-null  float64
 6   cust_request_tn            32022 non-null  float64
 7   tn                         32022 non-null  float64
 8   periodo_fecha              32022 non-null  object 
 9   mes                        32022 non-null  int64  
 10  meses_historia_prod        32022 non-null  int64  
 11  meses_historia_cliente     32022 non-null  int64  
 12  cliente_estrella           32022 non-null  int64  
 13  prod_cust                  32022 non-null  obj

In [12]:
df_sellout.isna().sum()

product_id                      0
periodo                         0
customer_id                     0
cero_ventas                     0
plan_precios_cuidados           0
cust_request_qty                0
cust_request_tn                 0
tn                              0
periodo_fecha                   0
mes                             0
meses_historia_prod             0
meses_historia_cliente          0
cliente_estrella                0
prod_cust                       0
cat1                            0
cat2                            0
cat3                            0
brand                           0
sku_size                        0
producto_estrella               0
temp_media                      0
temp_max_media                  0
temp_min_media                  0
IPC                             0
promedio_mens_dolar_venta       0
catastrofe                      0
tn_mas_2                     2384
dif_cust_request_tn             0
dif_cust_request_tn_porc        0
prod_cust_tn_l

In [13]:
df_sellout.columns

Index(['product_id', 'periodo', 'customer_id', 'cero_ventas',
       'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn',
       'periodo_fecha', 'mes', 'meses_historia_prod', 'meses_historia_cliente',
       'cliente_estrella', 'prod_cust', 'cat1', 'cat2', 'cat3', 'brand',
       'sku_size', 'producto_estrella', 'temp_media', 'temp_max_media',
       'temp_min_media', 'IPC', 'promedio_mens_dolar_venta', 'catastrofe',
       'tn_mas_2', 'dif_cust_request_tn', 'dif_cust_request_tn_porc',
       'prod_cust_tn_lag_1', 'prod_cust_tn_lag_2', 'prod_cust_tn_lag_3',
       'prod_cust_tn_lag_4', 'prod_cust_tn_lag_5'],
      dtype='object')

In [14]:
len(df_sellout.columns)

34